In [4]:
%%capture
!pip install transformers

In [5]:
from tensorflow.keras import backend as K
K.clear_session()

In [6]:
import gc
import tensorflow as tf 

# Custom Callback To Include in Callbacks List At Training Time
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

#Assign GPU

In [7]:
import tensorflow as tf

# Verifica le GPU disponibili

gpus = tf.config.list_physical_devices('GPU')
print("GPU disponibili:", gpus)

# Imposta il device di default su GPU
tf.config.set_visible_devices(gpus[0], 'GPU')

GPU disponibili: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Short characteristics

In [ ]:
import pandas as pd 

car = pd.read_csv("drive/MyDrive/BPM/cleaned_data/characteristics_description.csv")
filtered_car = car.dropna(subset=['description'], inplace=False)



# Read csv file 

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

from sklearn.model_selection import train_test_split
from transformers import TrainingArguments
import nltk


#df = pd.read_csv("drive/MyDrive/BPM/cleaned_data/filtered.csv")

#magic try

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Remove NA from completion 

In [ ]:
path = 'drive/MyDrive/BPM/'
df = pd.read_csv(path+'cleaned_data/characteristics_description.csv')
filtered_df = df.dropna(subset=['description'], inplace=False)
filtered_df.to_csv(path + 'filtered_characteristics_description.csv', index=False)



In [ ]:
import os
print(os.getcwd())
path = 'drive/MyDrive/BPM/'
filtered_df = pd.read_csv(path+'filtered_characteristics_description.csv')

/content


In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
bs = 2

# Tokenizer

In [ ]:
from transformers import GPT2Tokenizer

#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
max_length = 1024



In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Product description class 

In [ ]:
SPECIAL_TOKENS_DICT = {
    'pad_token': '<pad>',
    'additional_special_tokens': ['<context>', '<slogan>'],
}

tokenizer.add_special_tokens(SPECIAL_TOKENS_DICT)
#model.resize_token_embeddings(len(tokenizer))
# Show the full list of special tokens:
print(tokenizer.special_tokens_map)

{'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<pad>', 'additional_special_tokens': ['<context>', '<slogan>']}


In [ ]:

import torch
from torch.utils.data import Dataset
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

class ProductDataset(Dataset):

    def __init__(self, characteristics_list, descriptions_list, tokenizer, max_length=786):

        characteristics_tkn = tokenizer.additional_special_tokens_ids[0]
        description_tkn = tokenizer.additional_special_tokens_ids[1]
        pad_tkn = tokenizer.pad_token_id
        eos_tkn = tokenizer.eos_token_id
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attention_masks = []
        i = 0

        for description, characteristics in zip(descriptions_list, characteristics_list):

          input_text = f"Characteristics: {characteristics}\nDescription: {description}"

              # Tokenizzazione dell'input text
              encoding = tokenizer.encode_plus(
                  text=input_text,
                  padding="max_length",
                  truncation=True,
                  max_length=max_length,
                  return_tensors="pt"
              )

          #Aggiunta di input_ids, output_ids e attention_masks alle rispettive liste
          self.input_ids.append(encoding['input_ids'].squeeze())
          self.attention_masks.append(encoding['attention_mask'].squeeze())
        
          i += 1

    def __len__(self):
      return len(self.input_ids)

    def __getitem__(self, idx):
      return self.input_ids[idx], self.attention_masks[idx]


# Sentence Splitting

In [ ]:
# the output is a list, where each element is a sentence of the original text
nltk.sent_tokenize(lower_text)


# Preprocess data

In [ ]:
import string
def preprocess_text(text, stopwords_filtered):
    text = text.strip().lower()
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stopwords_filtered]
    processed_text = ' '.join(filtered_tokens)
    return processed_text

# Define training and validation dataset

In [ ]:

keep_words = [';', ',']  # Caratteri da mantenere
default_stopwords = set(stopwords.words('english'))
punctuation = nltk.sent_tokenize(string.punctuation)
stopwords_filtered = [word for word in default_stopwords if word not in keep_words]

description = filtered_df['description'].apply(lambda x:preprocess_text(x, stopwords_filtered))
characteristics = filtered_df['characteristics'].tolist()

description = description.tolist()
print(characteristics[0])
print(description[0])

 Electronics Kelby Training DVD: Mastering Blend Modes in Adobe Photoshop CS5 By Corey Barker
kelby training dvd mastering blend modes adobe photoshop cs5 corey barker useful tool becoming familiar use blend modes adobe photoshop . serious mastering photoshop offer , mastering blend modes important mastering layers.in dvd tutorial , seasoned expert corey barker explores function blend modes variety scenarios image restoration , sharpening , adjustments , special effects much . since every project scenario different , corey encourages experiment blend modes giving skills confidence need .


In [ ]:
import csv
data = pd.DataFrame({"characteristics": characteristics, "description": description})
data.to_csv(path + "preprocessing.csv",index=False)

In [ ]:
data = data.dropna(subset=['description'], inplace=False)
data.to_csv(path + 'preprocessing.csv', index=False)
data.head()

,characteristics,description
0,Electronics Kelby Training DVD: Mastering Ble...,kelby training dvd mastering blend modes adobe...
1,Electronics Digital Organizer and Messenger,digital organizer messenger
2,Electronics CLIKR-5 Time Warner Cable Remote ...,clikr-5 ur5u-8780l remote control designed ope...
3,Electronics Rand McNally 528881469 7-inch Int...,"like award-winning predecessor , intelliroute ..."
4,Electronics Polaroid Pbm2200 PC / Gaming Ster...,use high quality headphones internet chatting ...


In [ ]:
path = 'drive/MyDrive/BPM/'
data = pd.read_csv(path +"preprocessing.csv")
characteristics = data['characteristics'][:100000]
description = data['description'][:100000]
characteristics.shape
description.shape

(100000,)

In [ ]:
from torch.utils.data import random_split
dataset = ProductDataset(characteristics, description, tokenizer, max_length=786)

print(dataset)
# Split into training and validation sets
train_size = int(0.75 * len(dataset)) 
val_size = len(dataset) - train_size
print(len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

f'There are {train_size} samples for training, and {val_size} samples for validation testing'

19
22
24
25
31
100000


'There are 75000 samples for training, and 25000 samples for validation testing'

In [ ]:
dataset[123]

tensor([[50258, 27828, 46905,  ..., 50257, 50257, 50257],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [   -1,    -1,    -1,  ...,    -1,    -1,    -1]])

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # Sampling for training is random
            batch_size = bs
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # Sampling for validation is sequential as the order doesn't matter.
            batch_size = bs 
        )

In [ ]:
import random
from transformers import GPT2LMHeadModel, GPT2Config
import numpy as np

# Loading the model configuration and setting it to the GPT2 standard settings.
configuration = GPT2Config.from_pretrained('distilgpt2', output_hidden_states=False)

# Create the instance of the model and set the token size embedding length
model = GPT2LMHeadModel.from_pretrained("distilgpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()

# This step is optional but will enable reproducible runs.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# We wil create a few variables to define the training parameters of the model
# epochs are the training rounds
# the warmup steps are steps at the start of training that are ignored
# every x steps we will sample the model to test the output

epochs = 2
warmup_steps = 1e2
sample_every = 100

In [ ]:
from transformers import AdamW
# AdamW is a class from the huggingface library, it is the optimizer we will be using, and we will only be instantiating it with the default parameters. 
optimizer = AdamW(model.parameters(),
                  lr = 5e-4,
                  eps = 1e-8
                )


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_linear_schedule_with_warmup

"""
Total training steps is the number of data points, times the number of epochs. 
Essentially, epochs are training cycles, how many times each point will be seen by the model. 
"""

total_steps = len(train_dataloader) * epochs

"""
We can set a variable learning rate which will help scan larger areas of the 
problem space at higher LR earlier, then fine tune to find the exact model minima 
at lower LR later in training.
"""
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

In [ ]:
import random
import time
import datetime

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    print(f'Beginning epoch {epoch_i + 1} of {epochs}')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every 100 batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print(f'Batch {step} of {len(train_dataloader)}. Loss:{batch_loss}. Time:{elapsed}')

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print(f'Example output: {tokenizer.decode(sample_output, skip_special_tokens=True)}')
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print(f'Average Training Loss: {avg_train_loss}. Epoch time: {training_time}')

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids,  
                             attention_mask = b_masks,
                             labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print(f'Validation loss: {avg_val_loss}. Validation Time: {validation_time}')

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print(f'Total training took {format_time(time.time()-total_t0)}')

Beginning epoch 1 of 2


RuntimeError: ignored

In [ ]:
for i in range(epochs):

    print('\n--- Starting epoch #{} ---'.format(i))

    model.train()

    # These 2 lists will keep track of the batch losses and batch sizes over one epoch:
    losses = []
    nums = []

    for xb in tqdm(train_dl, desc="Training"):
      # Move the batch to the training device:
      inputs = batch.to(device)

      # Call the model with the token ids, segment ids, and the ground truth (labels)
      outputs = model(inputs[:,0,:], token_type_ids=inputs[:,1,:], labels=inputs[:,2,:])

      # Add the loss and batch size to the list:
      loss = outputs[0]
      losses.append(loss.item())
      nums.append(len(xb))

      loss.backward()

      optimizer.step()
      model.zero_grad()

    # Compute the average cost over one epoch:
    train_cost = np.sum(np.multiply(losses, nums)) / sum(nums)


    # Now do the same thing for validation:

    model.eval()

    with torch.no_grad():
      losses = []
      nums = []

      for xb in tqdm(val_dl, desc="Validation"):
        inputs = batch.to(device)
        outputs = model(inputs[:,0,:], token_type_ids=inputs[:,1,:], labels=inputs[:,2,:])
        losses.append(outputs[0].item())
        nums.append(len(xb))

    val_cost = np.sum(np.multiply(losses, nums)) / sum(nums)

    print('\n--- Epoch #{} finished --- Training cost: {} / Validation cost: {}'.format(i, train_cost, val_cost))

In [ ]:
import torch
torch.cuda.empty_cache()

torch.cuda.memory_summary(device=None, abbreviated=False)

NameError: ignored

In [ ]:
import pandas as pd
import jsonlines
import json

In [ ]:
pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tqdm import tqdm_notebook as tqdm
import json
import pandas as pd
output_path ='drive/MyDrive/BPM/train/metadata.jsonl'
df = pd.read_csv('drive/MyDrive/BPM/metadata.csv')
df.head()
d = pd.DataFrame()
d= d.assign(file_name=df['image'])
d = d.assign(additional_feature=df['text'])
d.head()
with open(output_path, "w") as f:
  f.write(d.to_json(orient='records', lines=True, index= 'true'))


In [ ]:
from tqdm import tqdm_notebook as tqdm
import json
import pandas as pd
data = []
output_path ='drive/MyDrive/BPM/train/metadata.jsonl'

with open(output_path, 'r') as f:
    for l in tqdm(f):
        data.append(json.loads(l))

full = pd.DataFrame(data)
print(full)

<ipython-input-45-bcdda83a91f0>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for l in tqdm(f):


0it [00:00, ?it/s]

       file_name                                 additional_feature
0      img_0.jpg  barnes noble nook power kit carbon bnadpn31 el...
1      img_1.jpg  groovy protective stand cover custom designed ...
2      img_2.jpg  barnes noble ov hb adp universal power kit ele...
3      img_3.jpg  barnes noble b505360439 wren quote cover elect...
4      img_4.jpg  barnes noble b211 4023 johnson quote cover ele...
..           ...                                                ...
995  img_995.jpg  startech com mxt101hq 150 150 feet coax high r...
996  img_996.jpg  kodak max aa battery kaa 4 electronics camera ...
997  img_997.jpg  cisco air ant4941 aironet 2 2dbi 2 4ghz dipole...
998  img_998.jpg  startech com mxt101hq 200 200 feet coax high r...
999  img_999.jpg  allied telesyn centrecom mr815t utp 8 port min...

[1000 rows x 2 columns]


In [ ]:
from PIL import Image
import requests
from io import BytesIO

def from_url_to_img(url):
  try:
    response = requests.get(url)
    if response.status_code == 200:
      img = Image.open(BytesIO(response.content))
      return img
    else:
      print(f"Errore nella richiesta dell'immagine da {url}. Codice di stato: {response.status_code}")
      return None
  except:
    return None

full['image'] = full['image'].apply(from_url_to_img)

full.head()
full.to_csv("'drive/MyDrive/BPM/cleaned_data/filtered_metadata.csv")

Errore nella richiesta dell'immagine da http://ecx.images-amazon.com/images/I/51PRS2FQAHL._SY300_.jpg. Codice di stato: 404
Errore nella richiesta dell'immagine da http://ecx.images-amazon.com/images/I/316XANTCQEL._SY300_.jpg. Codice di stato: 404
Errore nella richiesta dell'immagine da http://ecx.images-amazon.com/images/I/31RpRxzLjsL._SY300_.jpg. Codice di stato: 404
Errore nella richiesta dell'immagine da http://ecx.images-amazon.com/images/I/51SRQX6GEFL._SY300_.jpg. Codice di stato: 404
Errore nella richiesta dell'immagine da http://ecx.images-amazon.com/images/I/31KNMQVD9BL.jpg. Codice di stato: 404
Errore nella richiesta dell'immagine da http://ecx.images-amazon.com/images/I/41F9SWGP2ML._SY300_.jpg. Codice di stato: 404
Errore nella richiesta dell'immagine da http://ecx.images-amazon.com/images/I/41Hh5L2JQnL._SX300_.jpg. Codice di stato: 404
Errore nella richiesta dell'immagine da http://ecx.images-amazon.com/images/I/31oqjiAPmhL._SX300_.jpg. Codice di stato: 404
Errore nella ric

In [ ]:
import pandas as pd 
df = pd.read_csv("drive/MyDrive/BPM/StableDiffusion/cleaned_data/metadata.csv")
df.head()

,Unnamed: 0,image,text
0,0,drive/MyDrive/BPM/StableDiffusion/train/img_0.jpg,barnes noble nook power kit carbon bnadpn31 el...
1,1,drive/MyDrive/BPM/StableDiffusion/train/img_1.jpg,groovy protective stand cover custom designed ...
2,2,drive/MyDrive/BPM/StableDiffusion/train/img_2.jpg,barnes noble ov hb adp universal power kit ele...
3,3,drive/MyDrive/BPM/StableDiffusion/train/img_3.jpg,barnes noble b505360439 wren quote cover elect...
4,4,drive/MyDrive/BPM/StableDiffusion/train/img_4.jpg,barnes noble b211 4023 johnson quote cover ele...


In [ ]:
output_path ='drive/MyDrive/BPM/cleaned_data/filtered_metadata.jsonl'
full.shape
full = full.dropna()
full.shape
with open(output_path, "w") as f:
  full.write(d.to_json(orient='records', lines=True, index= 'true'))

In [ ]:
import pandas as pd
import nltk
max_token = 22
nltk.download('punkt')  # Scarica i dati necessari per la tokenizzazione

def riduci_dimensione(stringa, max_token):
    tokens = nltk.word_tokenize(stringa)
    stringa_ridotta = ' '.join(tokens[:max_token])
    return stringa_ridotta
df = pd.read_csv("drive/MyDrive/BPM/prova/prova.csv")
imgs = list(df['image'])
txts = list(df['text'])
imgs = [riduci_dimensione(s, max_token) for s in imgs]
txts = [riduci_dimensione(s, max_token) for s in txts]
print(imgs[0], txts[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


drive/MyDrive/BPM/prova/img/img_0.jpg barnes noble nook power kit carbon bnadpn31 electronics ebook readers accessories power adapters barnes noble


In [ ]:
from accelerate.utils import write_basic_config
import os

write_basic_config()
os.environ['MODEL_NAME']="CompVis/stable-diffusion-v1-4"
os.environ['TRAIN_DIR']="drive/MyDrive/BPM/train"
os.environ['OUTPUT_DIR']="drive/MyDrive/BPM/output"

!accelerate launch drive/MyDrive/BPM/train_text_to_image.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --train_data_dir=$TRAIN_DIR \
  --resolution=64 --center_crop --random_flip \
  --train_batch_size=2 \
  --gradient_accumulation_steps=2 \
  --gradient_checkpointing \
  --mixed_precision="no" \
  --max_train_steps=150 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --lr_scheduler="constant" --lr_warmup_steps=0 \
  --output_dir=${OUTPUT_DIR}

Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.
2023-05-22 10:06:01.969164: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-22 10:06:10.360521: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Namespace(input_pertubation=0, pretrained_model_name_or_path='CompVis/stable-diffusion-v1-4', revision=None, dataset_name=None, dataset_config_name=None, train_data_dir='drive/MyDrive/BPM/train', image_column='image', caption_column='additional_feature', max_train_samples=None, validation_prompts=None, output_dir='drive/MyDrive/BPM/output', cache_dir=None, seed=None, resolution=64, center_crop=True, random_flip=True, train_batch_size=2, num_train_epochs=100, max_train_steps=150, gradient_accumulation_steps=2, gradient_checkpointing=True, learning_rate=1e-05, scale_lr=

In [ ]:
import pandas as pd
df = pd.read_csv("drive/MyDrive/BPM/train/metadata.csv")
df.drop('Unnamed: 0', inplace=True, axis=1)
df.head()
df.to_csv("drive/MyDrive/BPM/train/metadata.csv")

In [ ]:
!pip install torchvision
!pip install transformers
!pip install datasets
!pip install ftfy
!pip install tensorboard
!pip install Jinja2
!pip install accelerate
!pip install diffusers
!pip install git+https://github.com/huggingface/diffusers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-itas3f7c
  Running command git clone

In [ ]:
import pandas as pd 
output_path = 'drive/MyDrive/BPM/train/metadata.jsonl'
df = pd.read_csv("drive/MyDrive/BPM/metadata.csv")
# df.head()
df.rename(columns={'image':'file_name', 'text':'additional_feature'}, inplace=True)
df.columns
with open(output_path, "w") as f:
  f.write(d.to_json(orient='records', lines=True, index= 'true'))


In [ ]:
import csv 
import json 

def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            jsonArray.append(row)
  
    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(jsonArray, indent=4)
        jsonf.write(jsonString)
          
csvFilePath = r'drive/MyDrive/BPM/train/metadata.csv'
jsonFilePath = r'drive/MyDrive/BPM/train/metadata.json'
csv_to_json(csvFilePath, jsonFilePath)

#Custom diffusion

In [ ]:
!pip install clip-retrieval 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.2/353.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0

In [ ]:
pip install -r drive/MyDrive/BPM/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.9/934.9 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 7.0.0
    Uninstalling pyarrow-7.0.0:
      Successfully uninstalled pyarrow-7.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
auto

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [ ]:
!python drive/MyDrive/BPM/retrieve.py --class_prompt cat --class_data_dir drive/MyDrive/BPM/real_reg/samples_cat --num_class_images 200

python3: can't open file '/content/drive/MyDrive/BPM/retrieve.py': [Errno 2] No such file or directory


In [ ]:
import os
os.environ['MODEL_NAME']="CompVis/stable-diffusion-v1-4"
os.environ['OUTPUT_DIR']="drive/MyDrive/BPM/output_dir"
os.environ['INSTANCE_DIR']="drive/MyDrive/BPM/data_1/cat"

!accelerate launch drive/MyDrive/BPM/train_custom_diffusion.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --class_data_dir=./real_reg/samples_cat/ \
  --with_prior_preservation --real_prior --prior_loss_weight=1.0 \
  --class_prompt="cat" --num_class_images=200 \
  --instance_prompt="photo of a <new1> cat"  \
  --resolution=512  \
  --train_batch_size=2  \
  --learning_rate=1e-5  \
  --lr_warmup_steps=0 \
  --max_train_steps=250 \
  --scale_lr --hflip  \
  --modifier_token "<new1>" 

Traceback (most recent call last):
  File "/content/drive/MyDrive/BPM/retrieve.py", line 20, in <module>
    from clip_retrieval.clip_client import ClipClient
ModuleNotFoundError: No module named 'clip_retrieval'
/bin/bash: accelerate: command not found


In [ ]:
!pip install diffusers
!pip install git+https://github.com/huggingface/diffusers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-f5vcxfe3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-f5vcxfe3
  Resolved https://github.com/huggingface/diffusers to commit c13dbd5c3a53017d27de35ad77b8d57f04c8ec7c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.17.0.dev0-py3-none-any.whl size=1010825 sha256=d36f140052a9301f345ea5619e01b95e043dfc6456f1cd6963107e2bf09c1a75
  Stored in directory: /tmp/pip-ephem-wheel-cache-257hsvuy/wheels/f7/7d/99/d361489e5762e3464b3811bc629e94cf5bf5ef44dd5c3c4d52
Successfully built diffusers
  Attempting uninstall: diffusers
    F